# Link communities
For this assignment we will take a look at link communities and how they differ from node communities. In order to do this we will use the algorithm discussed in the reading ("Link communities reveal multiscale complexity in networks") and link community video from canvas.

A small python module has been prepared that will allow you to use the link community algorithm with Anaconda and Python 3.5. To install the module you want to open a terminal or shell and use:

```
pip install git+https://github.com/Nathaniel-Rodriguez/linkcom.git
```

This will install the package locally in your Anaconda site-packages directory (the same place where the `conda` command would install new packages). If you have trouble installing the package you can just copy the .py files from the [github repository](https://github.com/Nathaniel-Rodriguez/linkcom/tree/master/linkcom) and put them in your working directory.

To use the package you can do (note if you download instead of install you will have to import each file individual):

In [1]:
import linkcom

The code has been adapted so that it will work with networkx graphs. The link communities algorithm requires simple undirected graphs to use. This means there can't be any self-loops or parallel edges. However, you can use weighted graphs.

In [2]:
# First lets import networkx
import networkx as nx

# And generate a new graph
my_graph = nx.erdos_renyi_graph(100, 0.1)

# We need to make sure this is a graph of type Graph
print(type(my_graph).__name__)

Graph


If the graph you load in isn't of type Graph (it maybe a Multigraph or DiGraph), it is easy to convert it to one:

In [3]:
my_graph = nx.Graph(my_graph)

## Using linkcom
Now lets call the `cluster` method in `linkcom` to custer the links of the graph. The `cluster` method takes several optional arguments:

```
linkcom.cluster(nx_graph, threshold=None, is_weighted=False, weight_key='weight', dendro_flag=False, to_file=False, basename="clustering", delimiter='\t')
```

`Threshold`: sets the cut-off for the dendrogram.

`is_weighted`: can be `True` or `False` depending upon whether the graph has weights or not. Set this to `True` if the graph is weighted.

`weight_key`: specifies what attribute the edges have that has weight values. In networkx it is convention that this key be set to `weight`. Most functions in networkx will assume this is the key. This is also the default value for the cluster method.

`dendro_flag`: specifies whether to return the dendrogram (only applicable if the graph is unweighted and no threshold is given).

`to_file`: specifies whether to write the outputs to file. Several files will be written and given names based on `basename` with elements separated by `delimiter`. 

These outputs will be written to file:

```
Three text files with extensions .edge2comm.txt, .comm2edges.txt,
and .comm2nodes.txt store the communities.

edge2comm, an edge on each line followed by the community
id (cid) of the edge's link comm:
node_i <delimiter> node_j <delimiter> cid <newline>

comm2edges, a list of edges representing one community per line:
cid <delimiter> ni,nj <delimiter> nx,ny [...] <newline>

comm2nodes, a list of nodes representing one community per line:
cid <delimiter> ni <delimiter> nj [...] <newline>

The output filename contains the threshold at which the dendrogram
was cut, if applicable, or the threshold where the maximum
partition density was found, and the value of the partition 
density.

If no threshold was given to cut the dendrogram, a file ending with
'_thr_D.txt' is generated, containing the partition density as a
function of clustering threshold.

If the dendrogram option was given, two files are generated. One with
'.cid2edge.txt' records the id of each edge and the other one with
'.linkage.txt' stores the linkage structure of the hierarchical 
clustering. In the linkage file, the edge in the first column is 
merged with the one in the second at the similarity value in the 
third column.
```

The cluster method will return a tuple with different elements:

    If no threshold is given, then a tuple is returned with: (dict) dictionary with keys=edges and values=community membership, (float) best similarity, (float) best partition density, (list) partition density list.

    If dendro_flag is given (only applicable if no threshold), then a tuple is returned with: (dict) dictionary with keys=edges and values=community membership, (float) best similarity, (float) best partition density, (list) partition density list, (dict) keys=edges and values=community membership for original, (list) dendrogram.

    If threshold is given a tuple is returned with: (dict) dictionary with keys=edges and values=community membership, partition density at threshold.

You will mostly just be interested in using the dictionary which contains the community assignment data, which is always the first element of the tuple. Lets do a short example:

In [4]:
e2c, S, D, Dlist = linkcom.cluster(my_graph)

clustering...
computing similarities...
# D_max = 0.083900
# S_max = 0.000000


If we print `e2c` we will see that each edge has a community membership:

In [5]:
print(e2c)

{(1, 34): 56, (0, 86): 56, (66, 94): 56, (45, 98): 56, (12, 78): 56, (33, 55): 56, (5, 44): 56, (62, 87): 56, (16, 28): 56, (63, 76): 56, (38, 50): 56, (24, 66): 56, (30, 51): 56, (16, 50): 56, (13, 32): 56, (62, 64): 56, (36, 61): 56, (1, 75): 56, (14, 31): 56, (65, 89): 56, (20, 88): 56, (64, 88): 56, (13, 58): 56, (45, 48): 56, (26, 98): 56, (0, 17): 56, (64, 75): 56, (58, 94): 56, (18, 38): 56, (47, 78): 56, (38, 46): 56, (59, 93): 56, (44, 86): 56, (7, 98): 56, (30, 48): 56, (41, 97): 56, (39, 97): 56, (20, 53): 56, (3, 23): 56, (43, 98): 56, (94, 98): 56, (28, 53): 56, (93, 97): 56, (22, 39): 56, (29, 56): 56, (28, 32): 56, (26, 89): 56, (39, 47): 56, (6, 35): 56, (31, 51): 56, (16, 81): 56, (50, 90): 56, (90, 95): 56, (29, 76): 56, (2, 99): 56, (64, 95): 56, (13, 51): 56, (12, 61): 56, (43, 57): 56, (15, 16): 56, (8, 50): 56, (33, 99): 56, (14, 63): 56, (19, 70): 56, (18, 66): 56, (50, 56): 56, (46, 82): 56, (13, 26): 56, (3, 11): 56, (32, 81): 56, (14, 99): 56, (18, 92): 56, (5

Since this is a random graph, we expect there not to be any meaningful communities, and indeed they are all labelled `56` (the community ID), so there doesn't appear to be any link communities in the graph.

We can now readily take our results and put them back into our graph so that it can be saved and viewed in Gephi. We can do this using the [`set_edge_attributes`](https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.classes.function.set_edge_attributes.html) function in networkx. It works just like the `set_node_attributes` function from previuos assignments, but with edges instead:

In [7]:
# Put the link communities into the graph
nx.set_edge_attributes(my_graph, "linkcom", e2c)

# Save the graph to file
nx.write_gexf(my_graph, "my_graph.gexf")

Now if we were to open the graph in Gephi we should be able to use the link communities to color the graph edges. In Gephi you may need to click the little attribute type button so that edge attributes are set to [ranked rather than numeric](http://www.clementlevallois.net/gephi/tuto/en/gephi_cheat%20sheets_en.pdf). This is because we want to color the edges according to their membership and not with a gradient. Additionally, in order to keep the edge colors when saving the graph you will need to make sure the edge color in the Preview tab is set to `original`. Lastly, since you will be looking at link communities (which determine the node membership in link clustering) it will be helpful to increase the size of the edges in Gephi so the colors are more visible and so you can detect nodes that belong to multiple communities.

## The network science collaboration graph
You will be using the NetSci collaboration graph for your assignment. The nodes of the graph are people and links are formed between people who co-author a scientific paper together in network science. You can download it from [here](http://vlado.fmf.uni-lj.si/pub/networks/data/collab/netscience.htm). Make sure to read the graph description on that page. The graph has ~1500 nodes and is partly disconnected. If you have difficulty working with the full graph or trouble loading it into Gephi, you can use the largest connected component (which only has about 350 nodes). The largest connected component of a graph can be returned from networkx using:

```
largest_component=max(nx.connected_component_subgraphs(my_graph), key=len)
```

Most of the interesting stuff is happening around this component anyway. So you don't lose much except for the scientists and groups that decided to work alone.

Follow these steps for the assignment:

1. Load the Netsci graph and run the link communities clustering algorithm on it.
2. Save the link communities to the graph and save the graph to file.
3. In Gephi choose a good layout for the graph.
4. Run the modularity command to generate communities for the nodes.
5. Color the edges according to the link communities and the nodes according to the communities found by Gephi. Remember to take care in choosing the resolution parameter.
6. How well does link clustering do at detecting community structure? How well does Gephi's node modularity do at detecting community structure? What do you think the communities represent?
7. What are the similarities and differences between the communities detected by either algorithm?
8. Which authors have a prominent position in multiple communities? What do you think these author's roles are?
9. What other features do you notice about the graph that are captured with overlaping communities?
10. Save your visualization to file.
11. Once complete, submit a PDF document to Canvas that contains your responses and your graph visualization (since this is a larger graph feel free to crop the figure so that it only includes parts relevant to your responses).